In [1]:
import pandas as pd
from geopandas import GeoDataFrame, read_file
import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()
import warnings
warnings.simplefilter("ignore")

/home/arg/miniconda3/envs/movingpandas/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)



MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
executable : /home/arg/miniconda3/envs/movingpandas/bin/python
machine    : Linux-5.15.0-113-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.8.4
GDAL data dir: /home/arg/miniconda3/envs/movingpandas/share/gdal
PROJ       : 9.3.1
PROJ data dir: /home/arg/miniconda3/envs/movingpandas/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.2.1
fiona      : 1.9.6
numpy      : 1.26.4
shapely    : 2.0.3
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.8.3
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : 1.2


In [2]:
import csv
import os
import shutil
import xml.etree.ElementTree as ET
from datetime import timedelta
import numpy as np
from dateutil import parser

global trajectory_id
def generate_grid_and_path(x_min, x_max, x_n, y_min, y_max, y_n):
    """
    Generates a grid and a path based on the given parameters.

    Parameters:
    - x_min (float): The minimum x-coordinate value.
    - x_max (float): The maximum x-coordinate value.
    - x_n (int): The number of points in the x-direction.
    - y_min (float): The minimum y-coordinate value.
    - y_max (float): The maximum y-coordinate value.
    - y_n (int): The number of points in the y-direction.

    Returns:
    - path (list): A list of coordinate tuples representing the path.

    """
    x_values = np.linspace(x_min, x_max, x_n)
    y_values = np.linspace(y_min, y_max, y_n)
    X, Y = np.meshgrid(x_values, y_values)
    path = []

    for i in range(x_n):
        for j in range(y_n):
            if i % 2 == 0:
                path.append((X[j, i], Y[j, i]))
            else:
                path.append((X[-j - 1, i], Y[-j - 1, i]))

    if x_n % 2 == 0:
        for j in range(y_n):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    else:
        for j in range(y_n - 1, -1, -1):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    return path


def generate_circle_path(x_center, y_center, radius, num_points):
    """
    Generate a circular path.

    Parameters:
    - x_center (float): x-coordinate of the center of the circle.
    - y_center (float): y-coordinate of the center of the circle.
    - radius (float): radius of the circle.
    - num_points (int): number of points to generate on the circle.

    Returns:
    - path (list): list of (x, y) coordinates representing the circular path.
    """
    theta_values = np.linspace(0, 2*np.pi, num_points)
    path = [(x_center + radius * np.cos(theta), y_center + radius * np.sin(theta)) for theta in theta_values]
    return path

def parse_coordinates_str_list_method1(kml_path):
    """
    Parses a KML file and extracts the coordinates from the LineString elements.

    Args:
        kml_path (str): The path to the KML file.

    Returns:
        list: A list of coordinate strings extracted from the LineString elements.
    """
    tree = ET.parse(kml_path)
    root = tree.getroot()
    coordinates = None
    coordinates_str_list = []
    for placemark in root.findall(".//Placemark"):
        name = placemark.find("name")
        if name is not None :
            # Find the LineString element
            line_string = placemark.find(".//LineString")
            if line_string is not None:
                # Extract the coordinates
                coordinates = line_string.find("coordinates")
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())
    return coordinates_str_list

def parse_coordinates_str_list_method2(kml_path):
    """
    Parses a KML file and extracts the coordinates from the LineString elements.

    Args:
        kml_path (str): The path to the KML file.

    Returns:
        list: A list of coordinate strings extracted from the LineString elements.
    """
    namespaces = {
        'kml': 'http://www.opengis.net/kml/2.2',
        'gx': 'http://www.google.com/kml/ext/2.2',
        'atom': 'http://www.w3.org/2005/Atom'
    }
    
    tree = ET.parse(kml_path)
    root = tree.getroot()
    coordinates_str_list = []

    for placemark in root.findall(".//kml:Placemark", namespaces):
        name = placemark.find("kml:name", namespaces)
        if name is not None:
            line_string = placemark.find(".//kml:LineString", namespaces)
            if line_string is not None:
                coordinates = line_string.find("kml:coordinates", namespaces)
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())          
    return coordinates_str_list

def get_means(coordinates):
    """
    Calculate the mean x and y coordinates from a list of coordinates.

    Parameters:
    coordinates (list): A list of coordinate tuples, where each tuple contains x and y coordinates.

    Returns:
    tuple: A tuple containing the mean x and y coordinates.

    """
    mean_x = np.mean([coord[0] for coord in coordinates])
    mean_y = np.mean([coord[1] for coord in coordinates])
    return mean_x, mean_y


def write_to_csv(
    csv_file_path, coordinates, trajectory_id, tracker, base_timestamp, mean_x, mean_y, grid_size_half=10e-5, xy_n=10
):
    """
    Writes coordinates to a CSV file.

    Args:
        csv_file_path (str): The path to the CSV file.
        coordinates (list): A list of coordinate tuples (x, y, z).
        trajectory_id (int): The ID of the trajectory.
        tracker (str): The tracker name.
        base_timestamp (str): The base timestamp for generating timestamps.
        mean_x (float): The mean x-coordinate.
        mean_y (float): The mean y-coordinate.
        grid_size_half (float, optional): Half of the grid size. Defaults to 10e-5.
        xy_n (int, optional): The number of points in the x and y directions. Defaults to 10.
    """
    base_time = parser.parse(base_timestamp)
    with open(csv_file_path, "a", newline="") as csv_file:
        writer = csv.writer(csv_file, delimiter=";")

        path = generate_circle_path(mean_x, mean_y, 0.00005, 10)

        for i, coord in enumerate(path, start=len(coordinates) + 1):
            x, y = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id, tracker, timestamp]
            writer.writerow(row)


def convert_kml_to_csv(csv_dir, kml_path, base_timestamp="2008-12-11 04:42:14+00", hz=1, grid_size_multiplier=1):
    """
    Converts a KML file to a CSV file.

    Args:
        kml_path (str): The path to the KML file.

    Returns:
        None
    """
    global trajectory_id
    global count
    kml_name = kml_path.split("/")[-1].split(".")[0]

    #print(parse_corrdinates_str_list(kml_path))
    coordinates_str_list = parse_coordinates_str_list_method1(kml_path)
    if coordinates_str_list == []:
        coordinates_str_list = parse_coordinates_str_list_method2(kml_path)
    coordinates_str_list = coordinates_str_list[0].split(" ")[::hz]
    #check if coord can convert to float
    coordinates = [[(coord) for coord in coord_str.split(",")] for coord_str in coordinates_str_list]
    print(kml_name, count)
    count += 1
        
    coordinates = [[float(coord) for coord in coord_str.split(",")] for coord_str in coordinates_str_list]
    mean_x, mean_y = get_means(coordinates)
    #print(f"Min x: {min([coord[0] for coord in coordinates])}, Min y: {min([coord[1] for coord in coordinates])}")
    #print(f"Max x: {max([coord[0] for coord in coordinates])}, Max y: {max([coord[1] for coord in coordinates])}")
    xy_n = 61
    grid_size_half = 1 / 111_111 * xy_n * grid_size_multiplier # Convert to meter

    #print(f"Mean x: {mean_x}, Mean y: {mean_y}")

    #trajectory_id = 1
    #trajectory_id = trajectory_id + 1  ## 
    tracker = 19

    csv_file_path = os.path.join(csv_dir, f"ALL.csv")
    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
    
    write_to_csv(
        csv_file_path,
        coordinates,
        trajectory_id,
        tracker,
        base_timestamp,
        mean_x,
        mean_y,
        grid_size_half,
        xy_n=xy_n,
    )
    trajectory_id = trajectory_id + 1
    # shutil.copy(csv_file_path, os.path.expanduser("./data/"))
    #print(f"Copied to ./data/{kml_name}.csv")

def get_specific_filenames(directory, extension):
    """
    Get a list of filenames in a directory that have a specific extension.

    Parameters:
    directory (str): The directory path to search for filenames.
    extension (str): The specific file extension to filter filenames.

    Returns:
    list: A list of filenames that have the specified extension.
    """
    filenames = [filename for filename in os.listdir(directory) if filename.endswith(extension)]
    return filenames

select_dataset = 2
if select_dataset == 1:
    csv_dir = "./data/ardupilot-csv"
    kml_dir = "./data/ardupilot-logs/kmls"
    timestamps_txt = "./data/ardupilot-logs-timestamps.txt"
    hz = 5
    grid_size_multiplier = 4
elif select_dataset == 2:
    csv_dir = "./data/uav-csv"
    kml_dir = "./data/raw_kml"
    timestamps_txt = "./data/uav-dataset-timestamps.txt"
    hz = 1
    grid_size_multiplier = 1

trajectory_id = 1
count = 0
csv_file_path = os.path.join(csv_dir, f"ALL.csv")
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)
csv_file = open(csv_file_path, "a", newline="")
writer = csv.writer(csv_file, delimiter=";")
headers = ["X", "Y", "fid", "id", "sequence", "trajectory_id", "tracker", "t"]
writer.writerow(headers)
csv_file.close()

kml_pathes = [os.path.join(kml_dir, f) for f in os.listdir(kml_dir) if f.endswith('.kml')]
kml_pathes.sort()


time_get = True
if time_get:
    with open(timestamps_txt, "r") as f:
        base_timestamps = f.readlines()

    for kml_path, base_timestamp in zip(kml_pathes, base_timestamps):
        convert_kml_to_csv(csv_dir, kml_path, base_timestamp.strip(), hz, grid_size_multiplier)
else:
    for kml_path in kml_pathes:
        convert_kml_to_csv(csv_dir, kml_path)

飛行軌跡_20230716070634_R8039938565 0
飛行軌跡_20230716070703_R8059961112 1
飛行軌跡_20230716071226_R6234489650 2
飛行軌跡_20230716071856_R6311160153 3
飛行軌跡_20230716073025_R6965064834 4
飛行軌跡_20230716080034_R6227154717 5
飛行軌跡_20230716082229_R1490595758 6
飛行軌跡_20230716085739_R6069402900 7
飛行軌跡_20230716091811_R4619961334 8
飛行軌跡_20230716092437_R0005495281 9
飛行軌跡_20230716093404_R0096327773 10
飛行軌跡_20230716093620_R0116350320 11
飛行軌跡_20230716093937_R5312932297 12
飛行軌跡_20230716094712_R1213191908 13
飛行軌跡_20230716095616_R8203501684 14
飛行軌跡_20230716100222_R4123783842 15
飛行軌跡_20230716102536_R0858151057 16
飛行軌跡_20230716102703_R8086290524 17
飛行軌跡_20230716102926_R3989969820 18
飛行軌跡_20230716103102_R4044176903 19
飛行軌跡_20230716103234_R1935948910 20
飛行軌跡_20230716103347_R1589705053 21
飛行軌跡_20230716103528_R4066640323 22
飛行軌跡_20230716103652_R1557961390 23
飛行軌跡_20230716103815_R4895866181 24
飛行軌跡_20230716104003_R7260967600 25
飛行軌跡_20230716144101_R2399850234 26
飛行軌跡_20230716144227_R5323142096 27
飛行軌跡_20230716144414_R077216336

In [3]:
import geoviews as gv

# Define your custom TrajectoryPlotter
class CustomTrajectoryPlotter(mpd.trajectory_plotter._TrajectoryPlotter):
    def get_color(self, i):
        return self.MPD_PALETTE[i % len(self.MPD_PALETTE)]

# Override the hvplot method to use the custom plotter
def custom_hvplot(self, *args, **kwargs):
    return CustomTrajectoryPlotter(self, *args, **kwargs).hvplot()

mpd.TrajectoryCollection.hvplot = custom_hvplot
bing_maps_tile_source = gv.tile_sources.WMTS("http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1")
hvplot_defaults = {'tiles': bing_maps_tile_source, 'frame_height': 700, 'frame_width': 700, 'colorbar': True}

csv_file_path = os.path.join(csv_dir, "ALL.csv")
df = pd.read_csv(csv_file_path, delimiter=';')

num_trajectories = df['trajectory_id'].nunique()

# If there are more than 3999 trajectories, sample 3999
if num_trajectories > 3999:
    print('Exceed hvplot limitation (3999): ', num_trajectories)
    sampled_trajectories = df['trajectory_id'].drop_duplicates().sample(n=100, random_state=1)
    df = df[df['trajectory_id'].isin(sampled_trajectories)]

df['trajectory_id'] = pd.factorize(df['trajectory_id'])[0] + 1

traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5, .8], **hvplot_defaults)


Exceed hvplot limitation (3999):  6868


:Overlay
   .WMTS.I          :WMTS   [Longitude,Latitude]
   .Path.I          :Path   [Longitude,Latitude]
   .Path.II         :Path   [Longitude,Latitude]
   .Path.III        :Path   [Longitude,Latitude]
   .Path.IV         :Path   [Longitude,Latitude]
   .Path.V          :Path   [Longitude,Latitude]
   .Path.VI         :Path   [Longitude,Latitude]
   .Path.VII        :Path   [Longitude,Latitude]
   .Path.VIII       :Path   [Longitude,Latitude]
   .Path.IX         :Path   [Longitude,Latitude]
   .Path.X          :Path   [Longitude,Latitude]
   .Path.XI         :Path   [Longitude,Latitude]
   .Path.XII        :Path   [Longitude,Latitude]
   .Path.XIII       :Path   [Longitude,Latitude]
   .Path.XIV        :Path   [Longitude,Latitude]
   .Path.XV         :Path   [Longitude,Latitude]
   .Path.XVI        :Path   [Longitude,Latitude]
   .Path.XVII       :Path   [Longitude,Latitude]
   .Path.XVIII      :Path   [Longitude,Latitude]
   .Path.XIX        :Path   [Longitude,Latitude]
   .Path.XX         :Path   [Longitude,Latitude]
   .Path.XXI        :Path   [Longitude,Latitude]
   .Path.XXII       :Path   [Longitude,Latitude]
   .Path.XXIII      :Path   [Longitude,Latitude]
   .Path.XXIV       :Path   [Longitude,Latitude]
   .Path.XXV        :Path   [Longitude,Latitude]
   .Path.XXVI       :Path   [Longitude,Latitude]
   .Path.XXVII      :Path   [Longitude,Latitude]
   .Path.XXVIII     :Path   [Longitude,Latitude]
   .Path.XXIX       :Path   [Longitude,Latitude]
   .Path.XXX        :Path   [Longitude,Latitude]
   .Path.XXXI       :Path   [Longitude,Latitude]
   .Path.XXXII      :Path   [Longitude,Latitude]
   .Path.XXXIII     :Path   [Longitude,Latitude]
   .Path.XXXIV      :Path   [Longitude,Latitude]
   .Path.XXXV       :Path   [Longitude,Latitude]
   .Path.XXXVI      :Path   [Longitude,Latitude]
   .Path.XXXVII     :Path   [Longitude,Latitude]
   .Path.XXXVIII    :Path   [Longitude,Latitude]
   .Path.XXXIX      :Path   [Longitude,Latitude]
   .Path.XL         :Path   [Longitude,Latitude]
   .Path.XLI        :Path   [Longitude,Latitude]
   .Path.XLII       :Path   [Longitude,Latitude]
   .Path.XLIII      :Path   [Longitude,Latitude]
   .Path.XLIV       :Path   [Longitude,Latitude]
   .Path.XLV        :Path   [Longitude,Latitude]
   .Path.XLVI       :Path   [Longitude,Latitude]
   .Path.XLVII      :Path   [Longitude,Latitude]
   .Path.XLVIII     :Path   [Longitude,Latitude]
   .Path.XLIX       :Path   [Longitude,Latitude]
   .Path.L          :Path   [Longitude,Latitude]
   .Path.LI         :Path   [Longitude,Latitude]
   .Path.LII        :Path   [Longitude,Latitude]
   .Path.LIII       :Path   [Longitude,Latitude]
   .Path.LIV        :Path   [Longitude,Latitude]
   .Path.LV         :Path   [Longitude,Latitude]
   .Path.LVI        :Path   [Longitude,Latitude]
   .Path.LVII       :Path   [Longitude,Latitude]
   .Path.LVIII      :Path   [Longitude,Latitude]
   .Path.LIX        :Path   [Longitude,Latitude]
   .Path.LX         :Path   [Longitude,Latitude]
   .Path.LXI        :Path   [Longitude,Latitude]
   .Path.LXII       :Path   [Longitude,Latitude]
   .Path.LXIII      :Path   [Longitude,Latitude]
   .Path.LXIV       :Path   [Longitude,Latitude]
   .Path.LXV        :Path   [Longitude,Latitude]
   .Path.LXVI       :Path   [Longitude,Latitude]
   .Path.LXVII      :Path   [Longitude,Latitude]
   .Path.LXVIII     :Path   [Longitude,Latitude]
   .Path.LXIX       :Path   [Longitude,Latitude]
   .Path.LXX        :Path   [Longitude,Latitude]
   .Path.LXXI       :Path   [Longitude,Latitude]
   .Path.LXXII      :Path   [Longitude,Latitude]
   .Path.LXXIII     :Path   [Longitude,Latitude]
   .Path.LXXIV      :Path   [Longitude,Latitude]
   .Path.LXXV       :Path   [Longitude,Latitude]
   .Path.LXXVI      :Path   [Longitude,Latitude]
   .Path.LXXVII     :Path   [Longitude,Latitude]
   .Path.LXXVIII    :Path   [Longitude,Latitude]
   .Path.LXXIX      :Path   [Longitude,Latitude]
   .Path.LXXX       :Path   [Longitude,Latitude]
   .Path.LXXXI      :P